In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/optiver_real_vol')

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')




Mounted at /content/drive
/bin/bash: line 1: nvidia-smi: command not found
Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


In [2]:

import pandas as pd
import numpy as np

import glob
import pickle

import matplotlib.pyplot as plt

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

from sklearn.linear_model import LinearRegression

### DATA MUNGING for HAR features extraction

In [3]:
"""
HAR models forecasting realized volatility

"""


def log_return(list_stock_prices):
    return np.log(list_stock_prices).diff()

def realized_volatility(series_log_return):
    return np.sqrt(np.sum(series_log_return**2))

def realized_quarticity(log_returns):
    return np.sum(log_returns**4) / len(log_returns)

def bipower_variation(log_returns):
    log_returns = log_returns.values
    bpv = 0
    for i in range(len(log_returns) - 1):
        bpv += np.abs(log_returns[i] * log_returns[i+1])
    return (np.sqrt(2 / np.pi)) ** (-2) * bpv

def positive_realized_semivariance(log_returns):
    log_returns = log_returns.values
    positive_returns = log_returns[log_returns > 0]
    return np.sum(positive_returns**2) / len(positive_returns)

def negative_realized_semivariance(log_returns):
    log_returns = log_returns.values
    negative_returns = log_returns[log_returns < 0]
    return np.sum(negative_returns**2) / len(negative_returns)


In [4]:
def book_wap1_HAR_estimator(subset_paths,train_target):

    book_wap1_HAR_df = pd.DataFrame()

    for path in subset_paths:
        print(path)
        st_id = int(path.split('/')[1].split('_')[1].split('=')[1])
        # stock ids in [103,18,31,37] have DIFFERENT length of time_id compared to target so we exclude them for now
        if st_id != -1:#not in [103,18,31,37,110]: # select stock id here
          df_book_data = pd.read_parquet(path)
          df_book_data['wap'] =(df_book_data['bid_price1'] * df_book_data['ask_size1']+df_book_data['ask_price1'] * df_book_data['bid_size1'])  / (
                                      df_book_data['bid_size1']+ df_book_data[
                                  'ask_size1'])
          df_book_data['log_ret_price'] = df_book_data.groupby(['time_id'])['wap'].apply(log_return).values
          df_book_data = df_book_data[~df_book_data['log_ret_price'].isnull()]
          train_st = df_book_data
          df_realized_vol_per_stock =  pd.DataFrame(train_st.groupby(['time_id'])['log_ret_price'].agg(realized_volatility)).rename(columns={'log_ret_price':'book_real_vol_price'})
          df_realized_quart_per_stock =  pd.DataFrame(train_st.groupby(['time_id'])['log_ret_price'].agg(realized_quarticity)).rename(columns={'log_ret_price':'book_real_quart_price'})
          df_realized_bpv_per_stock =  pd.DataFrame(train_st.groupby(['time_id'])['log_ret_price'].agg(bipower_variation)).rename(columns={'log_ret_price':'book_real_bpv_price'})
          df_realized_pos_semvar_per_stock =  pd.DataFrame(train_st.groupby(['time_id'])['log_ret_price'].agg(positive_realized_semivariance)).rename(columns={'log_ret_price':'book_real_pos_semvar_price'})
          df_realized_neg_semvar_per_stock =  pd.DataFrame(train_st.groupby(['time_id'])['log_ret_price'].agg(negative_realized_semivariance)).rename(columns={'log_ret_price':'book_real_neg_semvar_price'})

          target_st =train_target[train_target['stock_id']==st_id] # use when not called above
          target_st.index = [target_st["time_id"]] # use when not called above
          common_time_id = np.intersect1d(df_realized_vol_per_stock.index.values, target_st['time_id'].values) # ensure that only common time ids in trade_train and target are used.
          har_estimates_df = pd.DataFrame()
          har_estimates_df['book_real_vol_price'] = df_realized_vol_per_stock.loc[common_time_id,"book_real_vol_price"].values
          har_estimates_df['book_real_quart_price'] = df_realized_quart_per_stock.loc[common_time_id,"book_real_quart_price"].values
          har_estimates_df['book_real_bpv_price'] = df_realized_bpv_per_stock.loc[common_time_id,"book_real_bpv_price"].values
          har_estimates_df['book_real_pos_semvar_price'] = df_realized_pos_semvar_per_stock.loc[common_time_id,"book_real_pos_semvar_price"].values
          har_estimates_df['book_real_neg_semvar_price'] = df_realized_neg_semvar_per_stock.loc[common_time_id,"book_real_neg_semvar_price"].values
          har_estimates_df['target_vol'] = target_st.loc[common_time_id,"target"].values
          har_estimates_df['st_id'] = np.repeat(st_id,len(common_time_id))
          har_estimates_df['time_id'] = common_time_id

          book_wap1_HAR_df = pd.concat([book_wap1_HAR_df,har_estimates_df])

    return book_wap1_HAR_df

In [5]:

def trade_HAR_estimator(subset_paths,train_target):

    trade_HAR_df = pd.DataFrame()

    for path in subset_paths:
        print(path)
        st_id = int(path.split('/')[1].split('_')[1].split('=')[1])
        # stock ids in [103,18,31,37] have DIFFERENT length of time_id compared to target so we exclude them for now
        if st_id != -1:#not in [103,18,31,37,110]: # select stock id here
          train_st = pd.read_parquet(path)
          ## compute realized volality in the first 10 mins using trade execution price (instead of WAP)
          train_st['log_ret_price'] = train_st.groupby(by='time_id')['price'].apply(log_return).values
          train_st = train_st[~train_st['log_ret_price'].isnull()]
          df_realized_vol_per_stock =  pd.DataFrame(train_st.groupby(['time_id'])['log_ret_price'].agg(realized_volatility)).rename(columns={'log_ret_price':'real_vol_price'})
          df_realized_quart_per_stock =  pd.DataFrame(train_st.groupby(['time_id'])['log_ret_price'].agg(realized_quarticity)).rename(columns={'log_ret_price':'real_quart_price'})
          df_realized_bpv_per_stock =  pd.DataFrame(train_st.groupby(['time_id'])['log_ret_price'].agg(bipower_variation)).rename(columns={'log_ret_price':'real_bpv_price'})
          df_realized_pos_semvar_per_stock =  pd.DataFrame(train_st.groupby(['time_id'])['log_ret_price'].agg(positive_realized_semivariance)).rename(columns={'log_ret_price':'real_pos_semvar_price'})
          df_realized_neg_semvar_per_stock =  pd.DataFrame(train_st.groupby(['time_id'])['log_ret_price'].agg(negative_realized_semivariance)).rename(columns={'log_ret_price':'real_neg_semvar_price'})

          target_st =train_target[train_target['stock_id']==st_id] # use when not called above
          target_st.index = [target_st["time_id"]] # use when not called above
          common_time_id = np.intersect1d(df_realized_vol_per_stock.index.values, target_st['time_id'].values) # ensure that only common time ids in trade_train and target are used.
          har_estimates_df = pd.DataFrame()
          har_estimates_df['real_vol_price'] = df_realized_vol_per_stock.loc[common_time_id,"real_vol_price"].values
          har_estimates_df['real_quart_price'] = df_realized_quart_per_stock.loc[common_time_id,"real_quart_price"].values
          har_estimates_df['real_bpv_price'] = df_realized_bpv_per_stock.loc[common_time_id,"real_bpv_price"].values
          har_estimates_df['real_pos_semvar_price'] = df_realized_pos_semvar_per_stock.loc[common_time_id,"real_pos_semvar_price"].values
          har_estimates_df['real_neg_semvar_price'] = df_realized_neg_semvar_per_stock.loc[common_time_id,"real_neg_semvar_price"].values
          har_estimates_df['target_vol'] = target_st.loc[common_time_id,"target"].values
          har_estimates_df['st_id'] = np.repeat(st_id,len(common_time_id))
          har_estimates_df['time_id'] = common_time_id

          trade_HAR_df = pd.concat([trade_HAR_df,har_estimates_df])

    return trade_HAR_df

In [ ]:
"""
Create HAR features for trade data
"""

# os.chdir('/content/drive/MyDrive/optiver_real_vol/Datasets/optimus_GPU_data/data')
# subset_paths = glob.glob('trade_train.parquet/stock_id=*')
# train_target = pd.read_csv('train.csv')
# trade_HAR_vol_estimates_df = trade_HAR_estimator(subset_paths,train_target)
# os.chdir('/content/drive/MyDrive/optiver_real_vol/Datasets/HAR_features')
# pickle.dump(trade_HAR_vol_estimates_df, open('trade_HAR_vol_estimates_df.pkl', 'wb'))


'\nCreate HAR features for trade data\n'

In [ ]:
"""
Create HAR features for book data
"""

# os.chdir('/content/drive/MyDrive/optiver_real_vol/Datasets/optimus_GPU_data/data')
# subset_paths = glob.glob('book_train.parquet/stock_id=*')
# train_target = pd.read_csv('train.csv')
# book_wa1_HAR_vol_estimates_df = book_wap1_HAR_estimator(subset_paths,train_target)
# os.chdir('/content/drive/MyDrive/optiver_real_vol/Datasets/HAR_features')
# pickle.dump(book_wa1_HAR_vol_estimates_df, open('book_wa1_HAR_vol_estimates_df.pkl', 'wb'))


'\nCreate HAR features for book data\n'

In [ ]:
"""
reordering time_ids to correct order for trade wap1 data
"""

# os.chdir('/content/drive/MyDrive/optiver_real_vol/Datasets')
# correct_time_id_order = pickle.load(open('correct_time_id_order.pkl','rb'))

# os.chdir('/content/drive/MyDrive/optiver_real_vol/Datasets/HAR_features')
# trade_HAR_vol_estimates_df = pickle.load(open('trade_HAR_vol_estimates_df.pkl', 'rb'))


# # reorder time_ids to correct order

# def my_reorder_stock_in_df(df, time_ids_reordered):
#     common_values = [value for value in time_ids_reordered if value in df['time_id'].values]
#     df = df.set_index('time_id')
#     df = df.reindex(common_values)
#     df['time_id'] = df.index
#     return df

# # Assuming you have a dataframe called 'df' and an array of reordered time_ids called 'time_ids_reordered'
# HAR_trade_data_reordered = trade_HAR_vol_estimates_df.groupby('st_id').apply(my_reorder_stock_in_df, time_ids_reordered=correct_time_id_order).reset_index(drop=True)
# HAR_trade_data_reordered


# #### Load the dataset
# os.chdir('/content/drive/MyDrive/optiver_real_vol/Datasets/HAR_features')
# HAR_book_wap1_data_reordered = pickle.load(open('book_wap1_HAR_vol_estimates_reordered_df.pkl', 'rb'))
# temp_df = HAR_book_wap1_data_reordered.drop(columns=['target_vol', 'book_real_vol_price', 'book_real_quart_price', 'book_real_bpv_price', 'book_real_pos_semvar_price', 'book_real_neg_semvar_price'])
# temp_df

# ### fill the missing time ids in trade data to match with length of book data
# HAR_trade_data_reordered_df = pd.merge(temp_df,HAR_trade_data_reordered, left_on=['st_id','time_id'], right_on=['st_id','time_id'], how='left').ffill().bfill()
# HAR_trade_data_reordered_df

# #### save the dataset
# os.chdir('/content/drive/MyDrive/optiver_real_vol/Datasets/HAR_features')
# pickle.dump(HAR_trade_data_reordered_df,open('trade_HAR_vol_estimates_reordered_df.pkl', 'wb'))

'\nreordering time_ids to correct order for trade wap1 data\n'

In [ ]:
"""
reordering time_ids to correct order for book data
"""

# os.chdir('/content/drive/MyDrive/optiver_real_vol/Datasets')
# correct_time_id_order = pickle.load(open('correct_time_id_order.pkl','rb'))

# os.chdir('/content/drive/MyDrive/optiver_real_vol/Datasets/HAR_features')
# book_wap1_HAR_vol_estimates_full_df = pickle.load(open('book_wap1_HAR_vol_estimates_full_df.pkl', 'rb'))

# # reorder time_ids to correct order

# def my_reorder_stock_in_df(df, time_ids_reordered):
#     common_values = [value for value in time_ids_reordered if value in df['time_id'].values]
#     df = df.set_index('time_id')
#     df = df.reindex(common_values)
#     df['time_id'] = df.index
#     return df

# # Assuming you have a dataframe called 'df' and an array of reordered time_ids called 'time_ids_reordered'
# HAR_book_wap1_data_reordered = book_wap1_HAR_vol_estimates_full_df.groupby('st_id').apply(my_reorder_stock_in_df, time_ids_reordered=correct_time_id_order).reset_index(drop=True)
# HAR_book_wap1_data_reordered

# #### Load the dataset
# os.chdir('/content/drive/MyDrive/optiver_real_vol/Datasets/HAR_features')
# pickle.dump(HAR_book_wap1_data_reordered,open('book_wap1_HAR_vol_estimates_reordered_df.pkl', 'wb'))



'\nreordering time_ids to correct order for book data\n'

#### Training the HAR model for prediction

In [6]:
"""
load book wap1 HAR features
"""
os.chdir('/content/drive/MyDrive/optiver_real_vol/Datasets/HAR_features')
book_wap1_HAR_vol_estimates_full_df = pickle.load(open('book_wap1_HAR_vol_estimates_reordered_df.pkl', 'rb'))

book_wap1_HAR_vol_estimates_full_df = book_wap1_HAR_vol_estimates_full_df.rename(columns={'book_real_vol_price':'RV','book_real_quart_price': 'RQ', 'book_real_bpv_price': 'BPV', 'book_real_pos_semvar_price': 'RV_plus', 'book_real_neg_semvar_price': 'RV_minus'})
book_wa1_HAR_vol_estimates_df = book_wap1_HAR_vol_estimates_full_df.drop(columns=['target_vol','st_id',	'time_id'])
# book_wa1_HAR_vol_estimates_df



In [ ]:
book_wap1_HAR_vol_estimates_full_df

,RV,RQ,BPV,RV_plus,RV_minus,target_vol,st_id,time_id
0,0.007026,5.283978e-13,4.169624e-05,2.168871e-07,4.291532e-07,0.003267,0,4294
1,0.001474,1.827834e-15,9.639366e-07,1.829129e-08,1.684700e-08,0.001508,0,31984
2,0.000891,2.215738e-16,3.527636e-07,5.525326e-09,4.689908e-09,0.001326,0,31570
3,0.002395,3.136591e-15,3.698405e-06,2.412260e-08,2.120292e-08,0.002051,0,5666
4,0.001790,1.538828e-15,2.094555e-06,1.853102e-08,2.159987e-08,0.002364,0,29740
...,...,...,...,...,...,...,...,...
428927,0.007303,2.126995e-13,3.716398e-05,2.829112e-07,1.934924e-07,0.006360,126,24913
428928,0.005740,1.611561e-13,1.871075e-05,2.685819e-07,1.225794e-07,0.004802,126,15365
428929,0.004781,5.479353e-14,1.156482e-05,1.644787e-07,9.376588e-08,0.003397,126,29316
428930,0.006109,1.301908e-13,2.047856e-05,1.830165e-07,2.332819e-07,0.005479,126,32195


In [7]:
"""
load trade HAR features
"""

os.chdir('/content/drive/MyDrive/optiver_real_vol/Datasets/HAR_features')
trade_HAR_vol_estimates_full_df = pickle.load(open('trade_HAR_vol_estimates_reordered_df.pkl', 'rb'))

trade_HAR_vol_estimates_full_df = trade_HAR_vol_estimates_full_df.rename(columns={'real_vol_price':'RV','real_quart_price': 'RQ', 'real_bpv_price': 'BPV', 'real_pos_semvar_price': 'RV_plus', 'real_neg_semvar_price': 'RV_minus'})
trade_HAR_vol_estimates_df = trade_HAR_vol_estimates_full_df.drop(columns=['target_vol','st_id',	'time_id'])

## reorder columns
st_id = trade_HAR_vol_estimates_full_df['st_id']
time_id =  trade_HAR_vol_estimates_full_df['time_id']
trade_HAR_vol_estimates_full_df = trade_HAR_vol_estimates_full_df.drop(columns=['st_id',	'time_id'])
trade_HAR_vol_estimates_full_df['st_id'] = st_id
trade_HAR_vol_estimates_full_df['time_id'] = time_id
trade_HAR_vol_estimates_full_df

#trade_HAR_vol_estimates_df

,RV,RQ,BPV,RV_plus,RV_minus,target_vol,st_id,time_id
0,0.003655,3.051884e-12,1.002870e-05,1.617222e-06,5.219892e-07,0.003267,0,4294
1,0.000883,5.220685e-15,7.756356e-07,6.875064e-08,3.274881e-08,0.001508,0,31984
2,0.000814,2.121251e-15,7.144236e-07,3.885580e-08,1.955304e-08,0.001326,0,31570
3,0.000801,1.715036e-15,5.305640e-07,2.697830e-08,2.860591e-08,0.002051,0,5666
4,0.000861,2.641106e-15,6.454727e-07,5.277644e-08,2.054742e-08,0.002364,0,29740
...,...,...,...,...,...,...,...,...
428927,0.003977,2.920055e-13,1.380073e-05,4.879058e-07,2.522787e-07,0.006360,126,24913
428928,0.003725,1.794703e-13,1.546365e-05,4.518938e-07,1.273741e-07,0.004802,126,15365
428929,0.002514,9.457959e-14,5.698835e-06,1.847506e-07,1.988336e-07,0.003397,126,29316
428930,0.003218,1.270023e-13,1.119458e-05,2.978305e-07,1.758569e-07,0.005479,126,32195


In [24]:
"""
FULL original HAR CODE from https://github.com/jacob-hein/HAR-models-forecasting-realized-volatility-in-US-stocks/tree/main

"""

def data_set_scalar(DataSet, scalar=100000):
    DataSet.iloc[:, 2:5] *= scalar # only multiply	BPV,	RV_plus,	RV_minus because its small
    DataSet.iloc[:, 1:2] *= scalar*scalar # only  RQ by scalar twice because its even smaller
    return DataSet

#Forecast, Errors, Betas & Plots
def HAR(data, out_sample):
    RV = data['RV']
    RQ = data['RQ']
    BPV = data['BPV']
    RV_p = data['RV_plus']
    RV_m = data['RV_minus']


    nobs = len(RV)
    in_sample = nobs - out_sample
    print('nobs',nobs)
    print('in_sample',in_sample)
    print('out_sample',out_sample)

    outRV = RV[in_sample:]
    lag = 4 # 9 # 1 day = 9 time ids, 5 days (1 week) = 9*5 = 45 time ids
             # NOTE: lag has to be >= longest look back period. e.g. if  sum (mean) of past terms in HAR model below is 22 periods then  lag >= 22
             # in code below, lookback is 9 time ids and 45 time ids. i.e. 1 day and 5 days.

    all_predsA = np.zeros(out_sample)
    all_preds = np.zeros(out_sample)
    all_predsQ = np.zeros(out_sample)
    all_predsF = np.zeros(out_sample)
    all_predsC = np.zeros(out_sample)
    all_predsS = np.zeros(out_sample)
    all_predsJ = np.zeros(out_sample)

    all_betasA = np.zeros((out_sample, 4)) # variable for parameters, all_betasA axis 1 dim. is 1 larger than in XA due to intercept term
    all_betas = np.zeros((out_sample, 4))
    all_betasQ = np.zeros((out_sample, 5))
    all_betasF = np.zeros((out_sample, 7))
    all_betasC = np.zeros((out_sample, 4))
    all_betasS = np.zeros((out_sample, 5))
    all_betasJ = np.zeros((out_sample, 5))

    ## make single step predictions each time,t
    for t in range(out_sample):
        # Estimation
        #print(f'y first index {(lag + t + 1)}' )
        #print(f'in_sample {in_sample}' )
        y = RV[lag + t + 1: in_sample + t + 1]
        XA = np.zeros((in_sample - lag, 3)) # variable for data points
        X = np.zeros((in_sample - lag, 3))
        XQ = np.zeros((in_sample - lag, 4))
        XF = np.zeros((in_sample - lag, 6))
        XC = np.zeros((in_sample - lag, 3))
        XS = np.zeros((in_sample - lag, 4))
        XJ = np.zeros((in_sample - lag, 4))

        for i in range(in_sample - lag ):
            #print(f'x first index {(-1 + i + 1 + lag + t)}' )
            # AR(3)
            XA[i, 0] = RV[-1 + i + 1 + lag + t]
            XA[i, 1] = RV[-2 + i + 1 + lag + t]
            XA[i, 2] = RV[-3 + i + 1 + lag + t]

            # HAR
            X[i, 0] = RV[-1 + i + 1 + lag + t]
            X[i, 1] = np.mean(RV[-2 + i + 1 + lag + t: i + lag + t])
            X[i, 2] = np.mean(RV[-4 + i + 1 + lag + t: i + lag + t])

            # HARQ
            XQ[i, 0] = RV[-1 + i + 1 + lag + t]
            XQ[i, 1] = np.mean(RV[-2 + i + 1 + lag + t: i + lag + t])
            XQ[i, 2] = np.mean(RV[-4 + i + 1 + lag + t: i + lag + t])
            XQ[i, 3] = np.sqrt(RQ[-1 + i + 1 + lag + t]) * RV[-1 + i + 1 + lag + t]

            # HARQ-F
            XF[i, 0] = RV[-1 + i + 1 + lag + t]
            XF[i, 1] = np.mean(RV[-2 + i + 1 + lag + t: i + lag + t])
            XF[i, 2] = np.mean(RV[-4 + i + 1 + lag + t: i + lag + t])
            XF[i, 3] = np.sqrt(RQ[-1 + i + 1 + lag + t]) * RV[-1 + i + 1 + lag + t]
            XF[i, 4] = np.sqrt(np.mean(RQ[-2 + i + 1 + lag + t: i + lag + t])) * np.sqrt(np.mean(RV[-2 + i + 1 + lag + t: i + lag + t]))
            XF[i, 5] = np.sqrt(np.mean(RQ[-4 + i + 1 + lag + t: i + lag + t])) * np.sqrt(np.mean(RV[-4 + i + 1 + lag + t: i + lag + t]))

            # CHAR
            XC[i, 0] = BPV[-1 + i + 1 + lag + t]
            XC[i, 1] = np.mean(BPV[-2 + i + 1 + lag + t: i + lag + t])
            XC[i, 2] = np.mean(BPV[-4 + i + 1 + lag + t: i + lag + t])

            # SHAR
            XS[i, 0] = np.mean(RV[-2 + i + 1 + lag + t: i + lag + t])
            XS[i, 1] = np.mean(RV[-4 + i + 1 + lag + t: i + lag + t])
            XS[i, 2] = RV_p[-1 + i + 1 + lag + t]
            XS[i, 3] = RV_m[-1 + i + 1 + lag + t]

            # HAR-J
            XJ[i, 0] = RV[-1 + i + 1 + lag + t]
            XJ[i, 1] = np.mean(RV[-2 + i + 1 + lag + t: i + lag + t])
            XJ[i, 2] = np.mean(RV[-4 + i + 1 + lag + t: i + lag + t])
            XJ[i, 3] = max(RV[-1 + i + 1 + lag + t] - BPV[-1 + i + 1 + lag + t], 0)

        ### remove the last entry in Xs and ys during fitting to prevent lookahead bias
        XA = np.column_stack([np.ones(len(XA)), XA ]) # ones column concatenated to data for the intercept term.
        X = np.column_stack([np.ones(len(X)), X])
        XQ = np.column_stack([np.ones(len(XQ)), XQ])
        XF = np.column_stack([np.ones(len(XF)), XF])
        XC = np.column_stack([np.ones(len(XC)), XC])
        XS = np.column_stack([np.ones(len(XS)), XS])
        XJ = np.column_stack([np.ones(len(XJ)), XJ])

        # print('Full XA including prediction index = ' ,XA.shape)
        # print('Full y including prediction index = ',y.shape)

        ### remove the last entry in Xs and ys during fitting to prevent lookahead bias
        betasA = np.linalg.lstsq(XA[:-1,:], y[:-1], rcond=None)[0] # solve least squares
        betas = np.linalg.lstsq(X[:-1,:], y[:-1], rcond=None)[0]
        betasQ = np.linalg.lstsq(XQ[:-1,:], y[:-1], rcond=None)[0]
        betasF = np.linalg.lstsq(XF[:-1,:], y[:-1], rcond=None)[0]
        betasC = np.linalg.lstsq(XC[:-1,:], y[:-1], rcond=None)[0]
        betasS = np.linalg.lstsq(XS[:-1,:], y[:-1], rcond=None)[0]
        betasJ = np.linalg.lstsq(XJ[:-1,:], y[:-1], rcond=None)[0]

        # print('XA[:-1,:] with prediction index excluded = ' ,XA[:-1,:].shape)
        # print('y[:-1] with prediction index excluded = ',y[:-1].shape)

        b0A, b1A, b2A, b3A = betasA
        b0, b1, b2, b3 = betas
        b0Q, b1Q, b2Q, b3Q, b1Q_Q = betasQ
        b0F, b1F, b2F, b3F, b1F_Q, b2F_Q, b3F_Q = betasF
        b0C, b1C, b2C, b3C = betasC
        b0S, b1S_P, b2S, b3S, b1S_M = betasS
        b0J, b1J, b2J, b3J, bJ = betasJ

        # print('betasA',betasA)
        # print('b0A, b1A, b2A, b3A ', b0A, b1A, b2A, b3A )

        all_betasA[t] = betasA
        all_betas[t] = betas
        all_betasQ[t] = betasQ
        all_betasF[t] = betasF
        all_betasC[t] = betasC
        all_betasS[t] = betasS
        all_betasJ[t] = betasJ

        # Prediction at time-step t
        #print(f"prediction time index in original data, RV[] : {in_sample + t}")

        predA = b0A + b1A * XA[-1, 1] + b2A * XA[-1, 2] + b3A * XA[-1, 3]
        pred = b0 + b1 * X[-1, 1] + b2 * X[-1, 2] + b3 * X[-1, 3]
        predQ = b0Q + b1Q * XQ[-1, 1] + b2Q * XQ[-1, 2] + b3Q * XQ[-1, 3] + b1Q_Q * XQ[-1, 4]
        predF = b0F + b1F * XF[-1, 1] + b2F * XF[-1, 2] + b3F * XF[-1, 3] + b1F_Q * XF[-1, 4] + b2F_Q * XF[-1, 5] + b3F_Q * XF[-1, 6]
        predC = b0C + b1C * XC[-1, 1] + b2C * XC[-1, 2] + b3C * XC[-1, 3]
        predS = b0S + b1S_P * XS[-1, 1] + b2S * XS[-1, 2] + b3S * XS[-1, 3] + b1S_M * XS[-1, 4]
        predJ = b0J + b1J * XJ[-1, 1] + b2J * XJ[-1, 2] + b3J * XJ[-1, 3] + bJ * XJ[-1, 4]

        # Saving time-step t prediction within list for error computations
        all_predsA[t] = predA
        all_preds[t] = pred
        all_predsQ[t] = predQ
        all_predsF[t] = predF
        all_predsC[t] = predC
        all_predsS[t] = predS
        all_predsJ[t] = predJ

        # this step is essentially performed at the last out of sample prediction
        # to measure model goodness of fit. It is esentially same as the least squares done abvoe.
        # Regression at t=1 for Standard Errors before performing any out-of-sample forecasts
        if t == out_sample - 1:
            modelA = LinearRegression().fit(XA[:-1,:], y[:-1])
            model = LinearRegression().fit(X[:-1,:], y[:-1])
            modelQ = LinearRegression().fit(XQ[:-1,:], y[:-1])
            modelF = LinearRegression().fit(XF[:-1,:], y[:-1])
            modelC = LinearRegression().fit(XC[:-1,:], y[:-1])
            modelS = LinearRegression().fit(XS[:-1,:], y[:-1])
            modelJ = LinearRegression().fit(XJ[:-1,:], y[:-1])

            models_at_t_1 = {"modelA": modelA, "model": model, "modelQ": modelQ, "modelF": modelF, "modelC": modelC,
                             "modelS": modelS, "modelJ": modelJ}

            x_data = [XA[:-1,:], X[:-1,:], XQ[:-1,:], XF[:-1,:], XC[:-1,:], XS[:-1,:], XJ[:-1,:]]
            y_data = [y[:-1], y[:-1], y[:-1], y[:-1], y[:-1], y[:-1], y[:-1]]

            # Below we retrieve R^2 & Adjusted R^2, prior to out-of-sample forecasts
            r_squareds = {"R-squared": [], "Adj.R-squared": []}
            for X,y, val in zip( x_data, y_data, models_at_t_1.values() ):
                r_squareds["R-squared"].append(val.score(X, y))
                r_squareds["Adj.R-squared"].append(1 - (1 - val.score(X, y)) * ((in_sample - lag - 1) / (in_sample - lag - 3)))


    # Error Computations: Mean Squared Error & Mean Absolute Error
    AR_mse = np.mean((outRV - all_predsA) ** 2)
    AR_mae = np.mean(np.abs(outRV - all_predsA))

    HAR_mse = np.mean((outRV - all_preds) ** 2)
    HAR_mae = np.mean(np.abs(outRV - all_preds))

    HARQ_mse = np.mean((outRV - all_predsQ) ** 2)
    HARQ_mae = np.mean(np.abs(outRV - all_predsQ))

    HARQF_mse = np.mean((outRV - all_predsF) ** 2)
    HARQF_mae = np.mean(np.abs(outRV - all_predsF))

    CHAR_mse = np.mean((outRV - all_predsC) ** 2)
    CHAR_mae = np.mean(np.abs(outRV - all_predsC))

    SHAR_mse = np.mean((outRV - all_predsS) ** 2)
    SHAR_mae = np.mean(np.abs(outRV - all_predsS))

    HARJ_mse = np.mean((outRV - all_predsJ) ** 2)
    HARJ_mae = np.mean(np.abs(outRV - all_predsJ))

    errors_list = {"AR_mse": AR_mse, "AR_mae": AR_mae, "HAR_mse": HAR_mse, "HAR_mae": HAR_mae,
                   "HARQ_mse": HARQ_mse, "HARQ_mae": HARQ_mae, "HARQF_mse": HARQF_mse, "HARQF_mae": HARQF_mae,
                   "CHAR_mse": CHAR_mse, "CHAR_mae": CHAR_mae, "SHAR_mse": SHAR_mse, "SHAR_mae": SHAR_mae,
                   "HARJ_mse": HARJ_mse, "HARJ_mae": HARJ_mae}

    # Output formatting:
    ## the index of this dataframe is correct, same as out of sample prediction index in original data, RV[].
    output_df = pd.DataFrame({"stock_id": data["st_id"][in_sample:],"time_id": data["time_id"][in_sample:], "outRV": outRV, "all_predsA": all_predsA,
                              "all_preds": all_preds, "all_predsQ": all_predsQ, "all_predsF": all_predsF,
                              "all_predsC": all_predsC, "all_predsS": all_predsS, "all_predsJ": all_predsJ})

    betas_list = {"all_betasA": all_betasA, "all_betas": all_betas, "all_betasQ": all_betasQ,
                  "all_betasF": all_betasF, "all_betasC": all_betasC, "all_betasS": all_betasS,
                  "all_betasJ": all_betasJ}

    output_df_errors_betas = [output_df, errors_list, betas_list, models_at_t_1, r_squareds]

    return output_df_errors_betas



In [28]:
"""
Book wap1 HAR training model

"""

# out_sample=10 # 5
# start_time_id_index = 0
# end_time_id_index = 22*9 + out_sample # 5*9 + out_sample
unique_stock_ids = book_wap1_HAR_vol_estimates_full_df['st_id'].unique()

HAR_book_wap1_final_predictions = pd.DataFrame(index= book_wap1_HAR_vol_estimates_full_df.index, columns= ["outRV","all_predsA", "all_preds", "all_predsQ", "all_predsF", "all_predsC", "all_predsS", "all_predsJ"])
HAR_book_wap1_final_predictions[['st_id', 'time_id']] = book_wap1_HAR_vol_estimates_full_df[['st_id', 'time_id']]
HAR_book_wap1_final_predictions_errors_dict = {}

book_wap1_HAR_vol_estimates_full_df = data_set_scalar( book_wap1_HAR_vol_estimates_full_df )

training_length = 9 # 5*9 (use 6 days instead of 5 bcos of error), 22*9

for s in unique_stock_ids:
    print('stock id: ',s)

    book_wap1_data = book_wap1_HAR_vol_estimates_full_df[book_wap1_HAR_vol_estimates_full_df['st_id']==s]
    ind = book_wap1_data.index.values[:57]
    book_wap1_data.index = range(len(book_wap1_data))

    start_time_id_index = book_wap1_data.index[0]
    end_time_id_index = 56 #book_wap1_data.index[-1]
    data_length = end_time_id_index - start_time_id_index + 1
    out_sample = data_length - training_length

    #print(f'end_time_id_index - start_time_id_index + training_length + 1 :{ end_time_id_index - (start_time_id_index + training_length) + 1}')
    book_wap1_data = book_wap1_data.loc[start_time_id_index:end_time_id_index,:]

    HAR_book_wap1_output = HAR( book_wap1_data, out_sample = out_sample )

    ## save the prediction outputs.
    for col in ["outRV","all_predsA", "all_preds", "all_predsQ", "all_predsF", "all_predsC", "all_predsS", "all_predsJ"]:
        #print(f'HAR_book_wap1_output[0]["{col}"].values.shape : {HAR_book_wap1_output[0][col].values.shape}')
        print('len',len(ind), len(HAR_book_wap1_output[0][col].values))
        HAR_book_wap1_final_predictions.loc[ ind[training_length:] ,col] = HAR_book_wap1_output[0][col].values
        HAR_book_wap1_final_predictions_errors_dict[s] = HAR_book_wap1_output[1]


os.chdir('/content/drive/MyDrive/optiver_real_vol/Datasets/HAR_features')
HAR_book_wap1_final_predictions.to_parquet('HAR_book_wap1_final_predictions_init_time_ids.parquet')
pickle.dump(HAR_book_wap1_final_predictions_errors_dict,open('HAR_book_wap1_final_predictions_errors_dict_init_time_ids.pkl','wb'))

stock id:  0
nobs 57
in_sample 9
out_sample 48
len 57 48
len 57 48
len 57 48
len 57 48
len 57 48
len 57 48
len 57 48
len 57 48
stock id:  1
nobs 57
in_sample 9
out_sample 48
len 57 48
len 57 48
len 57 48
len 57 48
len 57 48
len 57 48
len 57 48
len 57 48
stock id:  2
nobs 57
in_sample 9
out_sample 48
len 57 48
len 57 48
len 57 48
len 57 48
len 57 48
len 57 48
len 57 48
len 57 48
stock id:  3
nobs 57
in_sample 9
out_sample 48
len 57 48
len 57 48
len 57 48
len 57 48
len 57 48
len 57 48
len 57 48
len 57 48
stock id:  4
nobs 57
in_sample 9
out_sample 48
len 57 48
len 57 48
len 57 48
len 57 48
len 57 48
len 57 48
len 57 48
len 57 48
stock id:  5
nobs 57
in_sample 9
out_sample 48
len 57 48
len 57 48
len 57 48
len 57 48
len 57 48
len 57 48
len 57 48
len 57 48
stock id:  6
nobs 57
in_sample 9
out_sample 48
len 57 48
len 57 48
len 57 48
len 57 48
len 57 48
len 57 48
len 57 48
len 57 48
stock id:  7
nobs 57
in_sample 9
out_sample 48
len 57 48
len 57 48
len 57 48
len 57 48
len 57 48
len 57 48
len 

In [ ]:
"""
trade price HAR training model

"""

# out_sample=10 # 5
# start_time_id_index = 0
# end_time_id_index = 22*9 + out_sample # 5*9 + out_sample
unique_stock_ids = trade_HAR_vol_estimates_full_df['st_id'].unique()

HAR_trade_price_final_predictions = pd.DataFrame(index= trade_HAR_vol_estimates_full_df.index, columns= ["tr_outRV","tr_all_predsA", "tr_all_preds", "tr_all_predsQ", "tr_all_predsF", "tr_all_predsC", "tr_all_predsS", "tr_all_predsJ"])
HAR_trade_price_final_predictions[['st_id', 'time_id']] = trade_HAR_vol_estimates_full_df[['st_id', 'time_id']]
HAR_trade_price_final_predictions_errors_dict = {}

trade_HAR_vol_estimates_full_df = data_set_scalar( trade_HAR_vol_estimates_full_df )

training_length = 9 # 5*9 (use 6 days instead of 5 bcos of error), 22*9

for s in unique_stock_ids:
    print('stock id: ',s)

    trade_price_data = trade_HAR_vol_estimates_full_df[trade_HAR_vol_estimates_full_df['st_id']==s]
    ind = trade_price_data.index.values[:57]
    trade_price_data.index = range(len(trade_price_data))

    start_time_id_index = trade_price_data.index[0]
    end_time_id_index = 56 #trade_price_data.index[-1]
    data_length = end_time_id_index - start_time_id_index + 1
    out_sample = data_length - training_length

    #print(f'end_time_id_index - start_time_id_index + training_length + 1 :{ end_time_id_index - (start_time_id_index + training_length) + 1}')
    trade_price_data = trade_price_data.loc[start_time_id_index:end_time_id_index,:]

    trade_price_output = HAR( trade_price_data, out_sample = out_sample )

    ## save the prediction outputs.
    for col1,col2 in zip(["tr_outRV","tr_all_predsA", "tr_all_preds", "tr_all_predsQ", "tr_all_predsF", "tr_all_predsC", "tr_all_predsS", "tr_all_predsJ"], ["outRV","all_predsA", "all_preds", "all_predsQ", "all_predsF", "all_predsC", "all_predsS", "all_predsJ"]):
        #print(f'trade_price_output[0]["{col1}"].values.shape : {trade_price_output[0][col2].values.shape}')
        HAR_trade_price_final_predictions.loc[ ind[training_length:], col1 ] = trade_price_output[0][col2].values
        HAR_trade_price_final_predictions_errors_dict[s] = trade_price_output[1]

os.chdir('/content/drive/MyDrive/optiver_real_vol/Datasets/HAR_features')
HAR_trade_price_final_predictions.to_parquet('HAR_trade_price_final_predictions_init_time_ids.parquet')
pickle.dump(HAR_trade_price_final_predictions_errors_dict,open('HAR_trade_price_final_predictions_errors_dict_init_time_ids.pkl','wb'))





In [ ]:
"""
Evaluate book wap1 HAR training model

"""

os.chdir('/content/drive/MyDrive/optiver_real_vol/Datasets/HAR_features')
HAR_book_wap1_final_predictions =  pd.read_parquet('HAR_book_wap1_final_predictions.parquet')
HAR_book_wap1_final_predictions_errors_dict = pickle.load(open('HAR_book_wap1_final_predictions_errors_dict.pkl','rb'))


In [ ]:
HAR_book_wap1_final_predictions.dropna()

,outRV,all_predsA,all_preds,all_predsQ,all_predsF,all_predsC,all_predsS,all_predsJ,st_id,time_id
54,0.002451,-0.005290,0.002503,0.002752,-0.003999,0.002191,0.002349,0.002503,0,24334
55,0.002361,-0.005840,0.002118,0.002304,0.002098,0.002246,0.003538,0.002118,0,19678
56,0.004381,0.002534,0.001516,0.001929,0.002240,0.001532,0.002685,0.001516,0,9583
57,0.000945,0.002245,0.000818,0.002564,0.002150,-0.000139,0.002844,0.000818,0,24010
58,0.001041,0.002835,-0.001167,0.003352,0.001503,0.001554,0.002747,-0.001167,0,229
...,...,...,...,...,...,...,...,...,...,...
428927,0.007303,0.005600,0.007756,0.006767,0.006758,0.007422,0.007075,0.007756,126,24913
428928,0.005740,0.007055,0.008716,0.006912,0.006912,0.008383,0.006072,0.008716,126,15365
428929,0.004781,0.006397,0.007508,0.006331,0.006048,0.005639,0.004444,0.007508,126,29316
428930,0.006109,0.005973,0.004847,0.004923,0.005290,0.004853,0.005027,0.004847,126,32195


In [ ]:
"""
Evaluate trade HAR training model

"""

os.chdir('/content/drive/MyDrive/optiver_real_vol/Datasets/HAR_features')
HAR_trade_price_final_predictions =  pd.read_parquet('HAR_trade_price_final_predictions.parquet')
HAR_trade_price_final_predictions_errors_dict = pickle.load(open('HAR_trade_price_final_predictions_errors_dict.pkl','rb'))


In [ ]:
HAR_trade_price_final_predictions.dropna()

,tr_outRV,tr_all_predsA,tr_all_preds,tr_all_predsQ,tr_all_predsF,tr_all_predsC,tr_all_predsS,tr_all_predsJ,st_id,time_id
54,1.023084e+07,-7.813377e+07,2.924452e+06,9.367104e+05,-6.035696e+08,-2.320635e+07,7.390856e+06,8.495468e+05,0,243340000000000
55,1.066911e+07,-9.165394e+07,1.088308e+07,1.127078e+07,1.072906e+07,9.274554e+06,1.211833e+07,1.098668e+07,0,196780000000000
56,1.855218e+07,1.676994e+07,1.329144e+07,1.378659e+07,1.407701e+07,1.094785e+07,1.465271e+07,1.354884e+07,0,95830000000000
57,8.192227e+06,1.505165e+07,1.921427e+06,7.322105e+06,2.889571e+06,7.847533e+06,7.550244e+06,5.012459e+06,0,240100000000000
58,7.294999e+06,1.658362e+07,6.227102e+06,1.625159e+06,1.540343e+07,6.364215e+06,2.233704e+06,1.436197e+06,0,2290000000000
...,...,...,...,...,...,...,...,...,...,...
428927,3.976532e+07,4.323559e+07,4.208820e+07,4.289276e+07,4.411589e+07,4.119137e+07,3.882986e+07,4.211428e+07,126,249130000000000
428928,3.725331e+07,4.616316e+07,4.686699e+07,4.537801e+07,4.276786e+07,4.409927e+07,4.774726e+07,4.612986e+07,126,153650000000000
428929,2.514378e+07,3.668812e+07,4.195735e+07,3.978539e+07,3.045097e+07,4.040545e+07,3.831974e+07,4.292850e+07,126,293160000000000
428930,3.217613e+07,3.594272e+07,3.412619e+07,4.340114e+07,2.581391e+06,3.252352e+07,3.582139e+07,2.656978e+07,126,321950000000000


In [ ]:
"""
Statistical significance test of model parameters
"""

# import pandas as pd
# import numpy as np
# from linearmodels import PanelOLS
# from linearmodels import IV2SLS
# from linearmodels.iv.results import IVModelComparison
# from statsmodels.tools.tools import add_constant
# from arch.bootstrap import IIDBootstrap
# from linearmodels.iv import IV2SLS



# def data_set_scalar(DataSet, scalar=100000):
#     DataSet.iloc[:, 1:] *= scalar
#     return DataSet

# def beta_table(forecast):
#     models = ['modelA', 'model', 'modelQ', 'modelF', 'modelC', 'modelS', 'modelJ']
#     robust_t_test = {}

#     for model in models:
#         robust_std_errs = coeftest(forecast[3][model], vcov=vcovHC(forecast[3][model], type="HC1"))
#         robust_t_test[model] = robust_std_errs

#     robust_t_test_matrix = np.vstack(list(robust_t_test.values()))[:, :2]
#     robust_t_test_table = np.full((10, 7), np.nan)
#     colnames = ["AR(3)", "HAR", "HARQ", "HARQ-F", "CHAR", "SHAR", "HAR-J"]
#     rownames = ["b0", "b1", "b2", "b3", "b1Q", "b2Q", "b3Q", "b1+", "b1-", "bJ"]

#     # Fill the table with coefficients and standard errors
#     c_ar = np.arange(len(forecast[3]['modelA'].params))
#     robust_t_test_table[c_ar, 0] = [f"{round(robust_t_test_matrix[i, 0], 4)} ({round(robust_t_test_matrix[i, 1], 4)})" for i in c_ar]

#     l_har = np.arange(len(forecast[3]['model'].params))
#     c_har = len(c_ar) + l_har
#     robust_t_test_table[l_har, 1] = [f"{round(robust_t_test_matrix[i, 0], 4)} ({round(robust_t_test_matrix[i, 1], 4)})" for i in c_har]

#     l_harq = np.arange(len(forecast[3]['modelQ'].params))
#     c_harq = len(c_har) + l_harq + 4
#     robust_t_test_table[l_harq, 2] = [f"{round(robust_t_test_matrix[i, 0], 4)} ({round(robust_t_test_matrix[i, 1], 4)})" for i in c_harq]

#     l_harqf = np.arange(len(forecast[3]['modelF'].params))
#     c_harqf = len(c_harq) + l_harqf + 8
#     robust_t_test_table[l_harqf, 3] = [f"{round(robust_t_test_matrix[i, 0], 4)} ({round(robust_t_test_matrix[i, 1], 4)})" for i in c_harqf]

#     l_char = np.arange(len(forecast[3]['modelC'].params))
#     c_char = len(l_harqf) + l_char + 13
#     robust_t_test_table[l_char, 4] = [f"{round(robust_t_test_matrix[i, 0], 4)} ({round(robust_t_test_matrix[i, 1], 4)})" for i in c_char]

#     l_shar = np.arange(len(forecast[3]['modelS'].params))
#     c_shar = len(l_char) + l_shar + 20
#     robust_t_test_table[[0, 2, 3, 7, 8], 5] = [f"{round(robust_t_test_matrix[i, 0], 4)} ({round(robust_t_test_matrix[i, 1], 4)})" for i in c_shar]

#     l_harj = np.arange(len(forecast[3]['modelJ'].params))
#     c_harj = len(l_shar) + l_harj + 24
#     robust_t_test_table[[0, 1, 2, 3, 9], 6] = [f"{round(robust_t_test_matrix[i, 0], 4)} ({round(robust_t_test_matrix[i, 1], 4)})" for i in c_harj]

#     # Append R^squareds to table
#     rsquareds = np.vstack([forecast[4].iloc[0, :], forecast[4].iloc[1, :]])
#     robust_t_test_table_w_rsquareds = np.vstack([robust_t_test_table, [np.nan]*7, rsquareds])
#     robust_t_test_table_w_rsquareds[11, :] = ["R^2", "Adj.R^2"] + [""] * 5

#     # Exporting Results
#     pd.DataFrame(robust_t_test_table_w_rsquareds, index=rownames + ["", ""], columns=colnames).to_excel("Results/Betas_w_all_rsquareds.xlsx", index=True, header=True)

#     return robust_t_test_table_w_rsquareds

# # Example usage
# data = pd.read_csv("data.csv")  # Replace "data.csv" with your actual data file
# RV_df = estimator(data)
# RV_df_scaled = data_set_scalar(RV_df)

In [ ]:
"""
# Plot ACF and PACF of RV to identify significant lags

# We don't need this because we just take lags 1 to 3 at max by default.
# """

# def compute_acf_pacf(unique_stock_ids,all_stock_train_pred_df,start_time_id_index,end_time_id_index):
#     ##### Autocorrelation and Partial Autocorrelation Plot EVERY individual stock
#     plt.close('all')
#     for s in unique_stock_ids[0:10]:
#         fig,ax = plt.subplots(2,1,figsize=(30,6))
#         stock_rv = all_stock_train_pred_df[all_stock_train_pred_df['st_id']==s].iloc[start_time_id_index:end_time_id_index]['RV']
#         #plot_acf(stock_rv, lags=50,ax=ax[0])
#         plot_pacf(stock_rv, lags=50,ax=ax[1])
#         #ax[0].set_title(f'Autocorrelation of stock {s} Real. Vol. ')
#         ax[1].set_title(f'Partial Autocorrelation of stock {s}  Real. Vol.')
#         #ax[0].set_xticks(range(0,50,5))
#         ax[1].set_xticks(range(0,50,5))
#         ax[1].set_xlabel('lags')
#         #ax[0].set_ylabel('ACF')
#         ax[1].set_ylabel('PACF')
#         #ax[0].grid(True)
#         ax[1].grid(True)
#         fig.show()
#     return

# unique_stock_ids = book_wap1_HAR_vol_estimates_full_df['st_id'].unique()

# ##### Autocorrelation and Partial Autocorrelation Plot EVERY individual stock
# start_time_id_index = 0
# end_time_id_index = 100
# compute_acf_pacf(unique_stock_ids,book_wap1_HAR_vol_estimates_full_df,start_time_id_index,end_time_id_index)

In [ ]:
"""
Copy of FULL original python code for reference

"""


# def data_set_scalar(DataSet, scalar=100000):
#     DataSet.iloc[:, 1:] *= scalar
#     return DataSet

# def HAR(data, out_sample=96, plot_scalar=1, extra_plots=False):
#     RV = data['RV']
#     RQ = data['RQ']
#     BPV = data['BPV']
#     RV_p = data['RV_plus']
#     RV_m = data['RV_minus']

#     nobs = len(RV)
#     in_sample = nobs - out_sample

#     outRV = RV[in_sample:]
#     lag = 22  # 22 days lag is equivalent to one month of trading days lag,

#     all_predsA = np.zeros(out_sample)
#     all_preds = np.zeros(out_sample)
#     all_predsQ = np.zeros(out_sample)
#     all_predsF = np.zeros(out_sample)
#     all_predsC = np.zeros(out_sample)
#     all_predsS = np.zeros(out_sample)
#     all_predsJ = np.zeros(out_sample)

#     all_betasA = np.zeros((out_sample, 4))
#     all_betas = np.zeros((out_sample, 4))
#     all_betasQ = np.zeros((out_sample, 5))
#     all_betasF = np.zeros((out_sample, 7))
#     all_betasC = np.zeros((out_sample, 4))
#     all_betasS = np.zeros((out_sample, 5))
#     all_betasJ = np.zeros((out_sample, 5))

#     for t in range(out_sample):
#         # Estimation
#         y = RV[(lag + t + 1):(in_sample + t)]
#         XA = np.zeros((in_sample - lag, 3))
#         X = np.zeros((in_sample - lag, 3))
#         XQ = np.zeros((in_sample - lag, 4))
#         XF = np.zeros((in_sample - lag, 6))
#         XC = np.zeros((in_sample - lag, 3))
#         XS = np.zeros((in_sample - lag, 4))
#         XJ = np.zeros((in_sample - lag, 4))

#         for i in range(in_sample - lag):
#             # AR(3) # three days back
#             print(RV[i - 1 + lag + t])
#             XA[i, 0] = RV[i - 1 + lag + t]
#             XA[i, 1] = RV[i - 2 + lag + t]
#             XA[i, 2] = RV[i - 3 + lag + t]

#             # HAR
#             X[i, 0] = RV[i - 1 + lag + t]
#             X[i, 1] = (1 / 5) * sum(RV[i - 5 + lag + t:i + lag + t])
#             X[i, 2] = (1 / 22) * sum(RV[i - 22 + lag + t:i + lag + t])

#             # HARQ
#             XQ[i, 0] = RV[i - 1 + lag + t]
#             XQ[i, 1] = (1 / 5) * sum(RV[i - 5 + lag + t:i + lag + t])
#             XQ[i, 2] = (1 / 22) * sum(RV[i - 22 + lag + t:i + lag + t])
#             XQ[i, 3] = (RQ[i - 1 + lag + t] ** (1 / 2)) * RV[i - 1 + lag + t]

#             # HARQ-F
#             XF[i, 0] = RV[i - 1 + lag + t]
#             XF[i, 1] = (1 / 5) * sum(RV[i - 5 + lag + t:i + lag + t])
#             XF[i, 2] = (1 / 22) * sum(RV[i - 22 + lag + t:i + lag + t])
#             XF[i, 3] = (RQ[i - 1 + lag + t] ** (1 / 2)) * RV[i - 1 + lag + t]
#             XF[i, 4] = (((1 / 5) * sum(RQ[i - 5 + lag + t:i + lag + t])) ** (1 / 2)) * (
#                     ((1 / 5) * sum(RV[i - 5 + lag + t:i + lag + t])) ** (1 / 2))
#             XF[i, 5] = (((1 / 22) * sum(RQ[i - 22 + lag + t:i + lag + t])) ** (1 / 2)) * (
#                     ((1 / 22) * sum(RV[i - 22 + lag + t:i + lag + t])) ** (1 / 2))

#             # CHAR
#             XC[i, 0] = BPV[i - 1 + lag + t]
#             XC[i, 1] = (1 / 5) * sum(BPV[i - 5 + lag + t:i + lag + t])
#             XC[i, 2] = (1 / 22) * sum(BPV[i - 22 + lag + t:i + lag + t])

#             # HARS
#             XS[i, 0] = RV_p[i - 1 + lag + t]
#             XS[i, 1] = (1 / 5) * sum(RV_p[i - 5 + lag + t:i + lag + t])
#             XS[i, 2] = (1 / 22) * sum(RV_p[i - 22 + lag + t:i + lag + t])
#             XS[i, 3] = (RV_m[i - 1 + lag + t] ** (1 / 2)) * RV_p[i - 1 + lag + t]

#             # HARQ-J
#             XJ[i, 0] = RV[i - 1 + lag + t]
#             XJ[i, 1] = (1 / 5) * sum(RV[i - 5 + lag + t:i + lag + t])
#             XJ[i, 2] = (1 / 22) * sum(RV[i - 22 + lag + t:i + lag + t])
#             XJ[i, 3] = (RQ[i - 1 + lag + t] ** (1 / 2)) * RV[i - 1 + lag + t]
#             XJ[i, 4] = (RV_m[i - 1 + lag + t] ** (1 / 2)) * RV_p[i - 1 + lag + t]

#         # AR(3)
#         betaA = np.linalg.inv(XA.T @ XA) @ XA.T @ y
#         all_betasA[t] = betaA

#         # HAR
#         beta = np.linalg.inv(X.T @ X) @ X.T @ y
#         all_betas[t] = beta

#         # HARQ
#         betaQ = np.linalg.inv(XQ.T @ XQ) @ XQ.T @ y
#         all_betasQ[t] = betaQ

#         # HARQ-F
#         betaF = np.linalg.inv(XF.T @ XF) @ XF.T @ y
#         all_betasF[t] = betaF

#         # CHAR
#         betaC = np.linalg.inv(XC.T @ XC) @ XC.T @ y
#         all_betasC[t] = betaC

#         # HARS
#         betaS = np.linalg.inv(XS.T @ XS) @ XS.T @ y
#         all_betasS[t] = betaS

#         # HARQ-J
#         betaJ = np.linalg.inv(XJ.T @ XJ) @ XJ.T @ y
#         all_betasJ[t] = betaJ

#         all_predsA[t] = XA[-1, :] @ betaA
#         all_preds[t] = X[-1, :] @ beta
#         all_predsQ[t] = XQ[-1, :] @ betaQ
#         all_predsF[t] = XF[-1, :] @ betaF
#         all_predsC[t] = XC[-1, :] @ betaC
#         all_predsS[t] = XS[-1, :] @ betaS
#         all_predsJ[t] = XJ[-1, :] @ betaJ

#     if extra_plots:
#         fig, axes = plt.subplots(3, 3, figsize=(15, 15))
#         ax1, ax2, ax3, ax4, ax5, ax6, ax7, ax8, ax9 = axes.flatten()

#         ax1.plot(all_predsA, label='AR(3)')
#         ax1.plot(outRV, label='RV_true')
#         ax1.legend()

#         ax2.plot(all_preds, label='HAR')
#         ax2.plot(outRV, label='RV_true')
#         ax2.legend()

#         ax3.plot(all_predsQ, label='HARQ')
#         ax3.plot(outRV, label='RV_true')
#         ax3.legend()

#         ax4.plot(all_predsF, label='HARQ-F')
#         ax4.plot(outRV, label='RV_true')
#         ax4.legend()

#         ax5.plot(all_predsC, label='CHAR')
#         ax5.plot(outRV, label='RV_true')
#         ax5.legend()

#         ax6.plot(all_predsS, label='HARS')
#         ax6.plot(outRV, label='RV_true')
#         ax6.legend()

#         ax7.plot(all_predsJ, label='HARQ-J')
#         ax7.plot(outRV, label='RV_true')
#         ax7.legend()

#         ax8.plot(all_predsA - outRV, label='AR(3)')
#         ax8.legend()

#         ax9.plot(all_preds - outRV, label='HAR')
#         ax9.legend()

#         plt.show()

#     return all_preds, all_predsQ, all_predsF, all_predsC, all_predsS, all_predsJ


# data = pd.read_csv("data.csv")  # Replace "data.csv" with your actual data file
# RV_df = estimator(data)
# predictions, predictions_Q, predictions_F, predictions_C, predictions_S, predictions_J = HAR(RV_df_scaled, extra_plots=True)


# import pandas as pd
# import numpy as np
# from linearmodels import PanelOLS
# from linearmodels import IV2SLS
# from linearmodels.iv.results import IVModelComparison
# from statsmodels.tools.tools import add_constant
# from arch.bootstrap import IIDBootstrap
# from linearmodels.iv import IV2SLS



# def data_set_scalar(DataSet, scalar=100000):
#     DataSet.iloc[:, 1:] *= scalar
#     return DataSet

# def beta_table(forecast):
#     models = ['modelA', 'model', 'modelQ', 'modelF', 'modelC', 'modelS', 'modelJ']
#     robust_t_test = {}

#     for model in models:
#         robust_std_errs = coeftest(forecast[3][model], vcov=vcovHC(forecast[3][model], type="HC1"))
#         robust_t_test[model] = robust_std_errs

#     robust_t_test_matrix = np.vstack(list(robust_t_test.values()))[:, :2]
#     robust_t_test_table = np.full((10, 7), np.nan)
#     colnames = ["AR(3)", "HAR", "HARQ", "HARQ-F", "CHAR", "SHAR", "HAR-J"]
#     rownames = ["b0", "b1", "b2", "b3", "b1Q", "b2Q", "b3Q", "b1+", "b1-", "bJ"]

#     # Fill the table with coefficients and standard errors
#     c_ar = np.arange(len(forecast[3]['modelA'].params))
#     robust_t_test_table[c_ar, 0] = [f"{round(robust_t_test_matrix[i, 0], 4)} ({round(robust_t_test_matrix[i, 1], 4)})" for i in c_ar]

#     l_har = np.arange(len(forecast[3]['model'].params))
#     c_har = len(c_ar) + l_har
#     robust_t_test_table[l_har, 1] = [f"{round(robust_t_test_matrix[i, 0], 4)} ({round(robust_t_test_matrix[i, 1], 4)})" for i in c_har]

#     l_harq = np.arange(len(forecast[3]['modelQ'].params))
#     c_harq = len(c_har) + l_harq + 4
#     robust_t_test_table[l_harq, 2] = [f"{round(robust_t_test_matrix[i, 0], 4)} ({round(robust_t_test_matrix[i, 1], 4)})" for i in c_harq]

#     l_harqf = np.arange(len(forecast[3]['modelF'].params))
#     c_harqf = len(c_harq) + l_harqf + 8
#     robust_t_test_table[l_harqf, 3] = [f"{round(robust_t_test_matrix[i, 0], 4)} ({round(robust_t_test_matrix[i, 1], 4)})" for i in c_harqf]

#     l_char = np.arange(len(forecast[3]['modelC'].params))
#     c_char = len(l_harqf) + l_char + 13
#     robust_t_test_table[l_char, 4] = [f"{round(robust_t_test_matrix[i, 0], 4)} ({round(robust_t_test_matrix[i, 1], 4)})" for i in c_char]

#     l_shar = np.arange(len(forecast[3]['modelS'].params))
#     c_shar = len(l_char) + l_shar + 20
#     robust_t_test_table[[0, 2, 3, 7, 8], 5] = [f"{round(robust_t_test_matrix[i, 0], 4)} ({round(robust_t_test_matrix[i, 1], 4)})" for i in c_shar]

#     l_harj = np.arange(len(forecast[3]['modelJ'].params))
#     c_harj = len(l_shar) + l_harj + 24
#     robust_t_test_table[[0, 1, 2, 3, 9], 6] = [f"{round(robust_t_test_matrix[i, 0], 4)} ({round(robust_t_test_matrix[i, 1], 4)})" for i in c_harj]

#     # Append R^squareds to table
#     rsquareds = np.vstack([forecast[4].iloc[0, :], forecast[4].iloc[1, :]])
#     robust_t_test_table_w_rsquareds = np.vstack([robust_t_test_table, [np.nan]*7, rsquareds])
#     robust_t_test_table_w_rsquareds[11, :] = ["R^2", "Adj.R^2"] + [""] * 5

#     # Exporting Results
#     pd.DataFrame(robust_t_test_table_w_rsquareds, index=rownames + ["", ""], columns=colnames).to_excel("Results/Betas_w_all_rsquareds.xlsx", index=True, header=True)

#     return robust_t_test_table_w_rsquareds

# # Example usage
# data = pd.read_csv("data.csv")  # Replace "data.csv" with your actual data file
# RV_df = estimator(data)
# RV_df_scaled = data_set_scalar(RV_df)






